In [1]:
import random
from data import get_df, num_sift_kp
def new_split():
    split = ["train"] * 200
    for i in range(0, 50):
        split[i * 4 + random.randint(0, 3)] = "test"
    return split
df = get_df()
num_split = 10
print("num_sift_kp={}, num_split={}".format(num_sift_kp, num_split))
for i in range(0, num_split):
    split_name = "split_" + str(i)
    df[split_name] = new_split()
    df[split_name] = df[split_name].astype("category")
# print(df.shape)
# print(df.dtypes)
# print(df.head())

num_sift_kp=25, num_split=10


In [2]:
from __future__ import print_function # Python 2
import numpy as np
import lib
from scipy.spatial.distance import cdist
sift_classifier = lib.SoftNearestClassClassifier(cdist)  # Euclidean distance by default
hue_classifier = lib.SoftNearestNeighbourClassifier(lambda x1, x2: np.minimum(x1, x2).sum())  # histogram intersection
correct_counts = np.empty(num_split, dtype=np.uint8)  # 1 * @num_split vector for # of true positives in each split
for i in range(0, num_split):
    sift_classifier.fit(lib.get_train_sift(df, i))
    hue_classifier.fit(lib.get_train_hue(df, i))
    predict_y = lib.test_sift_hue(df, i, sift_classifier, hue_classifier)
    correctness = np.arange(50) == predict_y
    incorrect_indices = np.nonzero(np.logical_not(correctness))[0]
    print("{} -> {}".format(incorrect_indices, predict_y[incorrect_indices]))
    correct_counts[i] = sum(correctness)
print(correct_counts)
accuracy = correct_counts.astype(np.float32) / 50
print("mean_accuracy={}, sigma={}".format(np.mean(accuracy), np.std(accuracy)))

[16 37] -> [33 39]
[ 2 11] -> [26  2]
[] -> []
[ 8 16] -> [ 4 33]
[ 8 26] -> [16 10]
[16 37] -> [33 39]
[39] -> [37]
[] -> []
[ 2 16 30 39 49] -> [21 33 41 31 46]
[ 2 39 49] -> [26 37 46]
[48 48 50 48 48 48 49 50 45 47]
mean_accuracy=0.962000012398, sigma=0.0274954624474
